# Arbol de decisión

Un árbol de decisión es un modelo de predicción, el cual utiliza distinos niveles de clasificación de datos con base en cierto valor (umbral). Divide los datos en grupos que sobrepasan o no ese umbral.

## Regresión

Este modelo utiliza todas las varibles y splits posibles, donde finalmente aplica el tipo de split con mayor reducción de varianza, con base en el error cuadrático medio.

$$\text{Reducción de varianza} = \text{Varianza total} - \left( \frac{N_1}{N} \times \text{Var}(G1) + \frac{N_2}{N} \times \text{Var}(G2) \right)$$

## Clasificación

De manera similar a la regresión, el calsificador utiliza todas las variables y umbrales posibles, eligiendo en este caso el umbral que maximice la impureza, con base en Gini o entropía. 

$$\text{Ganancia de Impureza} = \text{Impureza Nodo Padre} - \left( \frac{N_1}{N} \times \text{Impureza}(G1) + \frac{N_2}{N} \times \text{Impureza}(G2) \right)$$


# Random Forest

## Regresión

Utilizando bootstrap se hacen muestreos aleatorios sobre tus datos, con los cuales se entrena el modelo y se relaiza la predicción. Al final, cuando se tienen todas las predicciones, se calcula la media de esas predicciones, la cual es la predicción del random forest.

## Clasificación

En el modelo clasificador, en lugar de calcular la media de todas las predicciones, se utiliza la categoría que más repite (modal).

# Gradient boosting

Es un método que combina varios modelos "débiles" como los árboles de decisión, de modo que genera una predicción inicial y calcula error de la misma, para despues crear un modelo que trate de predecir este error, de modod que logre diminuirlo. Esto se repite iterativamente con un factor de aprendizaje hasta covnverger, muy parecido al decenso en gradiente. 

La predicción del árbol anterior es el promedio de los valores obtenidos por hoja. La diferencia entre el regresor y el clasificador es que en el clasificador usa los log-odds para predecir.

# Light GBM



# Catboost



# XGBoost

Es un modelo que consiste en sumar un nuevo árbol iterativamente a la predicción que ya se tiene.

$$\hat{y}_i^{(t)} = \hat{y}_i^{(t-1)} + f_t(x_i)$$

Donde:
- $\hat{y}_i^{(t)}$ es la predicción
- $f_t(x_i)$ es el nuevo árbol 

Con cada una de estas iteraciones su busca minimizar la contribución marginl del nuevo árbol a la pérdida total. Esta función de pérdida consiste de 2 partes: 

$$
\mathcal{L}^{(t)} =
\underbrace{
\sum_{i=1}^n l(y_i, \hat{y}_i^{(t-1)}) + \sum_{k=1}^{t-1} \Omega(f_k)
}_{\text{Parte ya construida (constante en esta iteración)}} +
\underbrace{
\sum_{i=1}^n \left[ l(y_i, \hat{y}_i^{(t-1)} + f_t(x_i)) - l(y_i, \hat{y}_i^{(t-1)}) \right] + \Omega(f_t)
}_{\text{Lo que añade el nuevo árbol $f_t$}}
$$

donde $\Omega$ es un factor de regularización que penaliza la complejidad de los árboles.

Ya que esta función de pérdida es compleja, se utiliza su expansión de Taylos de grado 2, ya que esta es fácil de optimizar.

$$f(x) \approx f(a) + f'(a)(x - a) + \frac{1}{2} f''(a)(x - a)^2$$

Aplicando el gradiente y hessiano con esta lógica, la función de pérdida queda de la manera:

$$\tilde{\mathcal{L}}^{(t)} = \sum_{i=1}^n \left[ g_i f_t(x_i) + \frac{1}{2} h_i f_t(x_i)^2 \right] + \Omega(f_t)$$

Ya que el modelo utiliza áboles de decisión de le agrega la constante $w_j$ a la función de pérdida, de modo que se logre regularizar:

$$
\mathcal{L}^{(t)} \approx \sum_{j=1}^{T} \left[
G_j w_j + \frac{1}{2} (H_j + \lambda) w_j^2
\right] + \gamma T
$$

Posteriormente, se minimiza el término por hoja respecto a $w_j$, obteniendo el output value.

$$\text{Output value} = w_j^* = -\frac{G_j}{H_j + \lambda}$$

Una vez que se tienen estos valores se calcula la mejora que hay en cada hoja por utilizar el óptimo:

$$\text{Similarity Score} = \frac{G_j^2}{H_j + \lambda}$$

Con base en este valor, se obtine el gain, para determinar si es útil realizar un split. Si el gain es positivo, se realiza el split, si es negativo se detiene.

$$\text{Gain} = \frac{1}{2} \left( \text{Similarity}_\text{izq} + \text{Similarity}_\text{der} - \text{Similarity}_\text{padre} \right) - \gamma$$

Finalmente con el árbol terminado se realiza la predicción final, sumando todos los árboles

$$\hat{y}_i = F_0(x_i) + \sum_{t=1}^{M} f_t(x_i)$$

# Comparación de modelos

| Característica              | **XGBoost**                                                | **LightGBM**                                               | **CatBoost**                                                  |
|-----------------------------|------------------------------------------------------------|-------------------------------------------------------------|----------------------------------------------------------------|
| **Velocidad**               | Rápido, pero más lento que LightGBM y CatBoost             | 🔥 Muy rápido gracias a histogramas y leaf-wise growth      | Rápido, aunque un poco más lento que LightGBM                  |
| **Precisión**               | Alta                                                       | Alta, a veces mejor con buen tuning                         | Muy alta, especialmente con categóricas                        |
| **Variables categóricas**   | ❌ No las maneja (requiere encoding manual)                | ❌ No las maneja (requiere encoding manual)                 | ✅ Soporte nativo + regularización secuencial                  |
| **Uso de memoria**          | Moderado                                                   | ✅ Muy eficiente (binning)                                   | Similar a XGBoost                                              |
| **Manejo de missing values**| ✅ Automático                                               | ✅ Automático                                                | ✅ Automático                                                   |
| **Soporte GPU**             | ✅ Sí (bastante estable)                                   | ✅ Sí (muy rápido)                                           | ✅ Sí (algo más limitado)                                      |
| **Instalación**             | Fácil (`pip install xgboost`)                             | Fácil (`pip install lightgbm`)                              | Un poco más pesada (`pip install catboost`)                   |
| **Documentación**           | Excelente                                                  | Buena                                                       | Muy buena                                                     |
| **Interacción con sklearn** | Muy buena                                                  | Muy buena                                                   | Muy buena                                                     |
| **Tolerancia al orden**     | ✅ Neutral                                                  | ✅ Neutral                                                   | ⚠️ Sensible (por codificación secuencial)                      |

Cuando usar cada modelo:

| Situación                                                  | Recomendación                                      |
|------------------------------------------------------------|----------------------------------------------------|
| Dataset tabular pequeño o mediano                          | ✅ XGBoost o CatBoost                               |
| Dataset grande, muchas variables numéricas                 | ✅ LightGBM                                         |
| Muchas variables categóricas sin preprocesamiento          | ✅ CatBoost (manejo nativo y robusto)              |
| Quieres algo robusto y estable con buen soporte            | ✅ XGBoost (muy probado en producción y Kaggle)     |
| Entrenamiento rápido con buen desempeño                    | ✅ LightGBM                                         |
| Quieres interpretabilidad con SHAP                         | ✅ Cualquiera, pero CatBoost da mejores resultados con categóricas |
| Necesitas buen rendimiento sin mucho tuning                | ✅ CatBoost (buenos defaults)                       |
| Ya tienes pipeline con OneHot/Target Encoding              | ✅ XGBoost o LightGBM                               |
| Tuning automático (Optuna, GridSearchCV, etc.)             | ✅ LightGBM (rápido y convergente)                  |
| Producción en sistemas legacy o APIs bien documentadas     | ✅ XGBoost (mayor madurez, más integración)         |
| Clasificación multi-label o problemas no estándar          | ✅ XGBoost (soporte más flexible)                   |

---

- **LightGBM** puede overfittear fácilmente → cuida `num_leaves` y `min_data_in_leaf`.
- **CatBoost** funciona muy bien con defaults y sin preprocessing.
- **XGBoost** es muy robusto y balanceado, ideal si ya tienes un pipeline con encoding hecho.

La estructura del árbol:

* XGBoost produce árboles más simétricos y balanceados.

* LightGBM produce árboles más profundos y desbalanceados si no se controla.

La precisión y riesgo de overfitting:

* Leaf-wise (LightGBM) puede encontrar mejores divisiones, pero se sobreajusta más fácil.

* Level-wise (XGBoost) es más estable, pero a veces menos preciso.